# ========================================

# Import Dependencies

# ========================================

In [1]:
import os
import csv
# import math
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
# import time
# import requests
# import datefinder

# # FOR SQL LITE# ========================================
# from sqlalchemy import create_engine
# from datetime import date

# # FOR PLOTTING
# import matplotlib.pyplot as plt
# from matplotlib import style
# style.use("fivethirtyeight")
# from matplotlib import rcParams
# rcParams['figure.figsize'] = 10, 8

# ========================================

# Step 1 - File Path for the CSV with Idaho Fire Data

# ========================================

## 1.1 File path for the Idaho Fire Data

In [2]:
# File path for the CSV File; where, ID = Idaho, frs = fires, pth = path

ID_frs_file_pth = os.path.join("Data", "fires_Idaho.csv")

## 1.2 List of Columns to Import from the Idaho Fire CSV

In [3]:
# List of columns to import

ID_frs_lst_Clmns = ["FOD_ID",
                    "FIRE_SIZE",
                    "FIRE_SIZE_CLASS",
                    "LATITUDE",
                    "LONGITUDE",
                    "COUNTY",
                    "DISCOVERY_DATE_CONVERTED",
                    "FIPS_NAME"]

## 1.3 Columns to Convert to a Datetime Data Type

In [4]:
parse_dates = ["DISCOVERY_DATE_CONVERTED"]

## 1.4 Import the Idaho Fire CSV

In [5]:
# Open the CSV Files, Convert to a Dataframe, and Save as a Variable, fires_Idaho_df
# Convert the Columne "DISCOVERY_DATE_CONVERTED" to a Datetime Datatype
# Reference: 
#     - Parsing Dates
#         - https://stackoverflow.com/questions/21269399/datetime-dtypes-in-pandas-read-csv
#     - Usecols
#         - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html


ID_frs_df = pd.read_csv(ID_frs_file_pth, usecols = ID_frs_lst_Clmns, parse_dates=parse_dates)

## 1.5 Check to Make Sure the Data Imported Correctly 

In [ ]:
# View the Data in the Dataframe to Make Sure it Imported Correctly
print(ID_frs_df.keys())
ID_frs_df.dtypes

## 1.6 Check to Make Sure FOD_ID is a Unique Idenitfier

If the Column "FOD_ID" is EQUAL to the Length of the "ID_frs_df" then PRINT "FOD_ID is a Unique Idenitfier"

Else If the Column "FOD_ID" is NOT EQUAL to the Length of the "ID_frs_df" then PRINT "NO!!!!!!! FOD_ID is NOT Unique Idenitfier"

In [6]:
if len(ID_frs_df) == len(pd.DataFrame(ID_frs_df["FOD_ID"].unique())):
    print("FOD_ID is a Unique Idenitfier")
elif len(ID_frs_df) == len(pd.DataFrame(ID_frs_df["FOD_ID"].unique())):
    print("NO!!!!!!! FOD_ID is NOT Unique Idenitfier")

FOD_ID is a Unique Idenitfier


In [13]:
len(ID_frs_df)

17360

# ========================================

# Step 2 - Import Historic Weather Data and Combine with the Idaho Fire Data (ID_frs_df)

# ========================================

## 2.1 - Import Meteostat Library

In [7]:
# Import Meteostat library and dependencies
from meteostat import Stations, Daily

## 2.2 - Pull the Data from the Meteostat library

## 2.2.1 - List of Columns for the Dataframe with Fire and Weather Data  Combined

### 2.2.1.1 - List of Columns from the Weather Data Dataframe

In [8]:
# List of columns to import from the Meteostat Library

ID_wthr_lst_Clmns = ["avg_temp", "prcp", "time"]

### 2.2.1.2 - Combine the List of Columns from the Weather Data and Fire Dataframe

In [9]:
# Combine the List of Columns from the Weather Data and Fire Dataframe

ID_cmbnd_wthr_frs_lst_Clmns = ID_wthr_lst_Clmns + ID_frs_lst_Clmns
print(ID_cmbnd_wthr_frs_lst_Clmns)

['avg_temp', 'prcp', 'time', 'FOD_ID', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'COUNTY', 'DISCOVERY_DATE_CONVERTED', 'FIPS_NAME']


## 2.2.2 - Loop Through Each Latitude and Longitude and Pull Weather the Data from the Meteostat Library and Combine it with the Idaho Fire Data

In [10]:
# Take the data and put it into a dataframe

# Idaho_county_Idaho_LatLong_df["LATITUDE"][59]

ID_cmbnd_wthr_frs_df = pd.DataFrame(columns = ID_cmbnd_wthr_frs_lst_Clmns)

for frs_row in ID_frs_df.index:
#         print(frs_row)
#         print(Idaho_county_Idaho_df_2["FOD_ID"][frs_row])
        # Set time period
        start = ID_frs_df["DISCOVERY_DATE_CONVERTED"][frs_row] - timedelta(days=3)
#         print(start)
        end = ID_frs_df["DISCOVERY_DATE_CONVERTED"][frs_row]
#         print(end)
        

        
        lat = ID_frs_df["LATITUDE"][frs_row]
#         print(Idaho_county_Idaho_df_2["LATITUDE"][frs_row])
        lon = ID_frs_df["LONGITUDE"][frs_row]
#         print(Idaho_county_Idaho_df_2["LONGITUDE"][frs_row])
        
        # Get closest weather station to Vancouver, BC
        stations = Stations()
        stations = stations.nearby(lat, lon)
        stations = stations.inventory('daily', (start, end))
        station = stations.fetch(1)

        # Get daily data for 2018 at the selected weather station
        data = Daily(station, start, end)
        data = data.aggregate() # Agg. Weekly data Commit out for daily, use 1M for monthly, 1W for weekly
        data = data.fetch()
        data = data.reset_index(level=0)
        
#         print(data["tavg"])
        
        for wthr_row in data.index:
#             print(data["tavg"][wthr_row])
#             print(data["prcp"][wthr_row])
#             print(data["time"][wthr_row])
            
            ID_cmbnd_wthr_frs_df = ID_cmbnd_wthr_frs_df.append({"FOD_ID": ID_frs_df["FOD_ID"][frs_row],
                                                                "FIRE_SIZE": ID_frs_df["FIRE_SIZE"][frs_row],
                                                                "FIRE_SIZE_CLASS": ID_frs_df["FIRE_SIZE_CLASS"][frs_row],
                                                                "COUNTY": ID_frs_df["COUNTY"][frs_row],
                                                                "DISCOVERY_DATE_CONVERTED": ID_frs_df["DISCOVERY_DATE_CONVERTED"][frs_row],
                                                                "FIPS_NAME": ID_frs_df["FIPS_NAME"][frs_row],
                                                                "LATITUDE": ID_frs_df["LATITUDE"][frs_row],
                                                                "LONGITUDE": ID_frs_df["LONGITUDE"][frs_row],
                                                                "avg_temp": data["tavg"][wthr_row],
                                                                "prcp": data["prcp"][wthr_row],
                                                                "time": data["time"][wthr_row]},
                                                                ignore_index = True)

In [11]:
ID_cmbnd_wthr_frs_df.head()

,avg_temp,prcp,time,FOD_ID,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,COUNTY,DISCOVERY_DATE_CONVERTED,FIPS_NAME
0,25.0,0.5,2005-07-13,155,0.1,A,44.488611,-111.256111,43,2005-07-16,Fremont
1,20.6,0.0,2005-07-14,155,0.1,A,44.488611,-111.256111,43,2005-07-16,Fremont
2,21.1,0.0,2005-07-15,155,0.1,A,44.488611,-111.256111,43,2005-07-16,Fremont
3,21.7,0.0,2005-07-16,155,0.1,A,44.488611,-111.256111,43,2005-07-16,Fremont
4,17.8,0.0,2005-06-29,172,0.1,A,42.736389,-112.384444,5,2005-07-02,Bannock


In [12]:
len(ID_cmbnd_wthr_frs_df)

69157

# Step 3 - Export the Data to a CSV File

In [14]:
# File path to export the CSV File
export_file_pth = os.path.join("Data", "Idaho_Fire_Weather_Data.csv")

ID_cmbnd_wthr_frs_df.to_csv(export_file_pth)